### Imports

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
from urllib.parse import urlencode
import time
import requests
import urllib3

### Setup

In [ ]:
urllib3.disable_warnings()
wslib = access_project_or_space()

CPD_HOST = "cpd-cpd.company.com"
PROJECT_ID = wslib.here.get_ID()
TOKEN = wslib.auth.get_current_token()

### Main API Call

In [3]:
def delete_dq_rule(token, project_id, rule_id):
    """
    Delete a single data quality rule using the Data Quality API
    
    Args:
        token (str): Authorization token
        project_id (str): The identifier of the project
        rule_id (str): The data quality rule identifier
    
    Returns:
        dict: Response with success status and details
    """
    try:
        
        # Build query parameters
        query_params = {}
        query_params['delete_output'] = 'false'
        query_params['cancel_unfinished_jobs'] = 'true'
        
        # Build URL
        url = f"https://{CPD_HOST}/data_quality/v3/projects/{project_id}/rules/{rule_id}"
        if query_params:
            url += f"?{urlencode(query_params)}"
        
        # Set headers
        headers = {
            'Authorization': f'Bearer {token}'
        }
        
        # Make the DELETE request
        response = requests.delete(url, headers=headers, verify=False)
        
        if response.status_code == 204:
            # Success - No Content
            return {
                'success': True,
                'message': f'Data quality rule {rule_id} deleted successfully',
                'rule_id': rule_id
            }
            
        elif response.status_code == 401:
            return {
                'success': False,
                'error': 'Unauthorized - Authorization missing, invalid, or expired',
                'status_code': response.status_code,
                'rule_id': rule_id
            }
            
        elif response.status_code == 403:
            return {
                'success': False,
                'error': 'Forbidden - No permission to delete the data quality rule',
                'status_code': response.status_code,
                'rule_id': rule_id
            }
            
        elif response.status_code == 404:
            return {
                'success': False,
                'error': 'Not Found - Data quality rule cannot be found',
                'status_code': response.status_code,
                'rule_id': rule_id
            }
            
        elif response.status_code == 500:
            return {
                'success': False,
                'error': 'Internal Server Error - Data quality rule cannot be deleted',
                'status_code': response.status_code,
                'rule_id': rule_id,
                'response': response.text
            }
            
        else:
            return {
                'success': False,
                'error': f"Unexpected HTTP {response.status_code}",
                'status_code': response.status_code,
                'rule_id': rule_id,
                'response': response.text
            }
            
    except ValueError as ve:
        return {
            'success': False,
            'error': f"Validation error: {str(ve)}",
            'rule_id': rule_id
        }
    except requests.exceptions.RequestException as re:
        return {
            'success': False,
            'error': f"Request error: {str(re)}",
            'rule_id': rule_id
        }
    except Exception as e:
        return {
            'success': False,
            'error': f"Unexpected error: {str(e)}",
            'rule_id': rule_id
        }

### Main Logic

In [ ]:
# List all assets of type Data Quality Rule
dq_rules = wslib.assets.list_assets("data_rule")

if len(dq_rules) == 0:
    print("No rules to delete")
else:
    print(f"\nDeleting {len(dq_rules)} data quality rules...")
    
    success_count = 0
    error_count = 0
    
    # Iterate through the rules
    for i, rule in enumerate(dq_rules, 1):
        rule_id = rule['asset_id']
        rule_name = rule['name']
        
        print(f"Deleting rule {i}/{len(dq_rules)}: {rule_id} - {rule_name}")
        
        # Call the delete_dq_rule function
        result = delete_dq_rule(
            token=TOKEN,
            project_id=PROJECT_ID,
            rule_id=rule_id
        )
        
        # Print results
        if result['success']:
            success_count += 1
            print(f"  ✓ Deleted successfully")
        else:
            error_count += 1
            print(f"  ✗ Failed: {result['error']}")
        
        # Small delay between calls
        if i < len(dq_rules):
            time.sleep(0.2)
    
    print(f"\nDeletion completed!")
    print(f"  Successfully deleted: {success_count}")
    print(f"  Errors: {error_count}")